## Part 2a: Implement an IR-based Chatbot

### Name: Yashaswini Joshi 

### Unique Name: yjoshi

### Kaggle Name: Yashaswini Joshi

##### Collaborated with Kevin Lee and Shivika Bisen

In [2]:
import pandas as pd
import csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/My Drive/SI 650/Homework 3

/content/drive/My Drive/SI 650/Homework 3


In [5]:
annotated_df = pd.read_csv("aggregated-hw3-ratings.train.csv", header=None)
annotated_df.columns= ['message_id','response_id','score']
annotated_df

,message_id,response_id,score
0,dtkk4zx,dtkmmnd,2.0
1,dtkk4zx,dtko1nq,1.0
2,dtkk4zx,dtkobtp,1.0
3,dtkk4zx,dtkoicj,2.0
4,dtkk4zx,dtkoy1k,2.0
...,...,...,...
17368,dur7pyz,dusf3in,1.0
17369,dur7pyz,dusgmup,0.0
17370,dur7pyz,dushdps,0.0
17371,dur7pyz,duslbrj,1.0


In [6]:
annotated_df.drop(columns='message_id',inplace=True)
annotated_df.head(10)

,response_id,score
0,dtkmmnd,2.0
1,dtko1nq,1.0
2,dtkobtp,1.0
3,dtkoicj,2.0
4,dtkoy1k,2.0
5,dtkp0mp,2.0
6,dtkp2ep,1.0
7,dtkpaic,2.0
8,dtkpo4c,1.0
9,dtkpu7o,0.0


In [7]:
chat_replies = pd.read_csv('chatbot-replies.tsv.gz', sep='\n\t|\t', engine='python',compression='gzip')
chat_replies

,message_id,message,response_id,response
0,dtjra8e,I developed Complex Regional Pain Syndrome aft...,dtjtpe4,That seems horrible. It has zero mobility? Wha...
1,dtjtpe4,That seems horrible. It has zero mobility? Wha...,dtjuwcb,I have about 2 degrees up and down and nothing...
2,dtjostm,This is a good poem! I'm sorry you seem to be ...,dtjp0ia,Thanks. I'm not really having a such a bad tim...
3,dtjp0ia,Thanks. I'm not really having a such a bad tim...,dtjpbdn,"Oh, ok! Well that's better, then! It's good yo..."
4,dtjriek,this is the best doki doki one I think I have ...,dtjsatq,Thanks! I hope it gets enough upvotes to get n...
...,...,...,...,...
2326024,duzm498,School shootings are a false flag operation to...,duzm5yo,"Yep, it works too because everyone has a soft ..."
2326025,duzm9lt,You dont have to buy it. You can even do it on...,duzmad2,I already used my trial unfortunately.
2326026,duzm61l,"I invested 8 years into a failed relationship,...",duzma8i,"Yep, and wow that must've been so painfull to ..."
2326027,duzm8sj,"Same, well back to refreshing, but twitch page...",duzmaqs,"Haha, you actual made me smile, what's wrong w..."


In [8]:
df = chat_replies.copy()

In [9]:
# Preprocessing of the columns response and message
df['response']= df.response.str.lower()
df['message']= df.message.str.lower()

df['response'] = df['response'].str.replace('[^A-Za-z0-9]+', ' ')
df['message'] = df['message'].str.replace('[^A-Za-z0-9]+', ' ')

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
df_copy = df.copy()

In [ ]:
import nltk
from nltk.corpus import stopwords
df_copy = df.copy()
df_copy['response'] = df_copy['response'].astype(str)
df_copy['message'] = df_copy['message'].astype(str)
df_copy['response'] = df_copy['response'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords.words('english'))]))
df_copy['message'] = df_copy['message'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords.words('english'))]))

In [51]:
#Loading test data
test = pd.read_csv("aggregated-hw3-rating.test.csv")
# renaming the column name to message_id so that it'll be easy to merge 
test = test.rename(columns={"messagd_id": "message_id"})
test

,message_id,response_id
0,dtmjt1b,dtmk5mt
1,dtmjt1b,dtml6kp
2,dtmjt1b,dtmlc7o
3,dtmjt1b,dtmm4wx
4,dtmjt1b,dtmpi9d
...,...,...
17275,duwy9ae,duxhh8h
17276,duwy9ae,duxjdqf
17277,duwy9ae,duxxxnf
17278,duwy9ae,duxyadw


In [38]:
# Merging the test data and the preprocessed data
testing = test.merge(df_copy,how='inner',on=['message_id'])
testing.drop(columns='response_id_x', inplace = True)
testing.rename(columns={"response_id_y": "response_id"}, inplace=True)

In [14]:
df10 = df.copy()
# Dropping the message ids which have less than 10 responses
df10 = df10[df10.message_id != 'dtncx90']
df10 = df10[df10.message_id != 'dtncx79']

In [15]:
# Messages of the messages ids with less than 10 responses
query1 = testing[(testing['message_id'] == 'dtncx90')].message.values[0]
query2 = testing[(testing['message_id'] == 'dtncx79')].message.values[0]

In [16]:
message_df = [x for x in df10.message]

In [17]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

In [18]:
def get_response1():
    responses = []
    tokenized_messages = [doc.split(" ") for doc in message_df]
    bm25 = BM25Okapi(tokenized_messages)
    
    tokenized_query = query1.split(" ")
    top_6 = bm25.get_top_n(tokenized_query, message_df, n=6)
    responses.append(top_6)
    return responses

In [19]:
message_1 = get_response1()

In [21]:
def get_response2():
    responses = []
    tokenized_messages = [doc.split(" ") for doc in message_df]
    bm25 = BM25Okapi(tokenized_messages)
    
    tokenized_query = query2.split(" ")
    top_6 = bm25.get_top_n(tokenized_query, message_df, n=9)
    responses.append(top_6)
    return responses

In [23]:
message_2 = get_response2()

In [24]:
query1_output = df10[(df10['message'] == 'best vikings technically three starting caliber qbs team worst vikings technically zero starting caliber qbs team')]

In [25]:
query1_output.message_id[query1_output.message_id=='du17tn9'] = 'dtncx90'

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


In [39]:
testing = pd.concat([testing,query1_output[:6]], axis=0)

In [40]:
for i in message_2:
  for j in i:
    msg = df10[(df10['message'] == j)]
    msg2 = pd.concat([msg], axis=0)
    testing = pd.concat([testing, msg], axis = 0)

In [41]:
testing.message_id[testing.message_id=='dtp228l'] = 'dtncx79'
testing.message_id[testing.message_id=='dtnwmpe'] = 'dtncx79'
testing.message_id[testing.message_id=='dty20pp'] = 'dtncx79'
testing.message_id[testing.message_id=='dugb799'] = 'dtncx79'
testing.message_id[testing.message_id=='duwenii'] = 'dtncx79'
testing.message_id[testing.message_id=='dtst3eq'] = 'dtncx79'
testing.message_id[testing.message_id=='dutr30k'] = 'dtncx79'
testing.message_id[testing.message_id=='dum9q3u'] = 'dtncx79'
testing.message_id[testing.message_id=='dubgwjl'] = 'dtncx79'

In [42]:
final_df = testing.drop_duplicates(subset=['response_id'])

In [43]:
responses = [x for x in final_df.response]

## Part 2b

In [48]:
dff = test.copy()
dff = dff.merge(chat_replies, how = 'inner', on = ['message_id'])
dff = dff.drop_duplicates(subset = ["message_id"])
query_df = [i for i in dff.message]

In [49]:
def get_bot_response():
    responses_list = []
    tokenized_messages = [doc.split(" ") for doc in responses]
    bm25 = BM25Okapi(tokenized_messages)
    
    for i in query_df:
        tokenized_query = i.split(" ")
        top_10 = bm25.get_top_n(tokenized_query, responses, n=10)
        responses.append(top_10)
    return responses

In [ ]:
bm25 = get_bot_response()

In [18]:
response_ids = []
for i in bm25:
    temp = []
    for j in i:
        r_id = df.response_id[df['response'] == j].values
        temp.append(r_id[0])
    response_ids.append(temp)

In [19]:
len(response_ids)

281

In [20]:
from pandas.core.common import flatten
lastleglist = list(flatten(response_ids))
len(lastleglist)

2810

In [21]:
from pandas import DataFrame
lastleg = DataFrame (lastleglist,columns=['reponse_id'])
lastleg

,reponse_id
0,dtoryq8
1,dtosti6
2,dtwzyj9
3,dtx6uce
4,du1co07
...,...
2805,dux8g0k
2806,dtoygnj
2807,dtsr7ji
2808,dtxyij9


In [23]:
response_id_list =[]
for i in response_ids:
  for j in i:
    response_id_list.append(j)

response_id_list

['dtoryq8',
 'dtosti6',
 'dtwzyj9',
 'dtx6uce',
 'du1co07',
 'du0zba3',
 'dts1b0x',
 'dtkxpgv',
 'dtm9vqv',
 'dtl0dcb',
 'dtkpylt',
 'dtkwg9j',
 'dtkxpgv',
 'dtkysxk',
 'dtktnnd',
 'dtkzwmj',
 'dtkm5r8',
 'dtv8bdr',
 'dtkuzyq',
 'dtmjx0e',
 'du0pxpf',
 'dtmfla0',
 'dui856q',
 'duxn5y1',
 'dtmouif',
 'du0jnro',
 'dtkwc5w',
 'dtnyy58',
 'dtqduzq',
 'dusqmaq',
 'dtnh1mw',
 'dtybfnb',
 'dtn237g',
 'dtwoz7p',
 'dtlwijh',
 'dtmypow',
 'duqc75x',
 'dusgoja',
 'dtmvbvq',
 'dtmmugk',
 'dtk9o81',
 'dtld4iq',
 'dtnrki9',
 'dtn47ua',
 'dtyld4u',
 'du1idcl',
 'dunrx2j',
 'dtzggbz',
 'dtlaf5l',
 'dtwl8yq',
 'dtu3z6r',
 'duwgqng',
 'dtque4y',
 'dttbhmk',
 'dtxv9ds',
 'duwwcfm',
 'duu95xd',
 'dtool7h',
 'dts00tz',
 'dtt0vtj',
 'dtlrliu',
 'dtmirwg',
 'duq46gr',
 'dtlnhl1',
 'duh9re5',
 'dtsz6p7',
 'dtld4j0',
 'dtlrjrf',
 'duxdffc',
 'dtl27jr',
 'dtp2cau',
 'dtkysay',
 'dtqzcmy',
 'dtobvo4',
 'dtw0r48',
 'dtzi9ne',
 'dtlnqib',
 'dtkqyfd',
 'dtkmbz3',
 'dtporiz',
 'du4n2ia',
 'dtmxb04',
 'dtlqhfu',
 'du

In [24]:
message_ids = [x for x in dff.message_id]
message_ids

['dtmjt1b',
 'dtklkv4',
 'dtmq9ph',
 'dtkqccy',
 'dtncx72',
 'dtm6r9c',
 'dtkfuo7',
 'dtkhafi',
 'dtkjb37',
 'dtkkoni',
 'dtkqazv',
 'dtn6ay8',
 'dtvqa6u',
 'dtw20ca',
 'dtvualg',
 'dtkchsw',
 'dtkhqfn',
 'dtl04cd',
 'dtm1vtl',
 'dtmfccp',
 'dtkizj2',
 'dtnqir6',
 'dtl5s6q',
 'dtna933',
 'dtla9w7',
 'dtlypab',
 'dtmt9j3',
 'dtkid9z',
 'dtmu86e',
 'dtmm8eu',
 'dtkl330',
 'dtq3psm',
 'dtq92a7',
 'dtq8uy3',
 'dtqeck0',
 'dtqsw3m',
 'dtku31b',
 'dtn50mc',
 'dtnva01',
 'dtk4396',
 'dtkqe8g',
 'dtl7b9f',
 'dtlq0u7',
 'dtlhayu',
 'dtn4qit',
 'dtn55qp',
 'dtrszmv',
 'dtrt0gd',
 'dtrydvw',
 'dtrtkkm',
 'dts0br6',
 'dtmpyff',
 'dtmjnix',
 'dtrsxwr',
 'dtrtofu',
 'dtrpeqf',
 'dtruged',
 'dtrsti0',
 'dtrqfkf',
 'dtrqh6z',
 'dtjpqg3',
 'dtkh9dz',
 'dtl7vbs',
 'dtlat1r',
 'dtm2sx2',
 'dtkiwvg',
 'dtkk739',
 'dtq50nb',
 'dtt73jt',
 'dtkqekk',
 'dtkho3a',
 'dtn522h',
 'dtkd7fp',
 'dtkf1wg',
 'dtmgp4z',
 'dtkl73v',
 'dtmq7rv',
 'dtkjbp3',
 'dtmtnpz',
 'dtlkui0',
 'dtmnluo',
 'dtkmp5e',
 'dtk2oge',
 'dt

In [25]:
message_id_list = []
for i in message_ids:
  for j in range(0,10):
    message_id_list.append(i)
    count = j

message_id_list

['dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtmjt1b',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtklkv4',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtmq9ph',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtkqccy',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtncx72',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtm6r9c',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkfuo7',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkhafi',
 'dtkjb37',
 'dtkjb37',
 'dtkjb37',
 'dt

In [27]:
kaggle = pd.DataFrame()
kaggle['message_id'] = message_id_list
kaggle['response_id'] = response_id_list
kaggle.to_csv('kaggle.csv',index=False)

In [ ]:
from google.colab import files
files.download("kaggle.csv")